## Import

In [7]:
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage

import pandas as pd
import numpy as np
import datetime
import os

## Load data

In [8]:
data_path = os.getcwd() + "\\datasets\\"

In [9]:
stock_df = pd.read_csv(data_path + "stock_df.csv")

In [10]:
stock_df.replace(np.NaN, 1, inplace = True)

In [11]:
stock_df.head()

,Date,3S_Close,AJ네트웍스_Close,AK홀딩스_Close,APS홀딩스_Close,AP시스템_Close,AP위성_Close,ARIRANG 200_Close,ARIRANG 200동일가중_Close,ARIRANG 200선물레버리지_Close,...,흥국화재우_Close,흥아해운_Close,희림_Close,히어로즈 TDF2030액티브_Close,히어로즈 TDF2040액티브_Close,히어로즈 TDF2050액티브_Close,히어로즈 글로벌리츠이지스액티브_Close,히어로즈 단기채권ESG액티브_Close,히어로즈 리츠이지스액티브_Close,힘스_Close
0,1994-03-05,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1994-03-07,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1994-03-08,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1994-03-09,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1994-03-10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


* date setting

In [13]:
start_date = "2001-01-01"
#start_date = "2021-01-01"
end_date = datetime.datetime.now().strftime("%Y-%m-%d")

In [14]:
use_df = stock_df.loc[stock_df.Date >= start_date, ~stock_df.columns.isin(['Date'])]

In [ ]:
mu = mean_historical_return(use_df.iloc[:, :5])
S = CovarianceShrinkage(use_df.iloc[:, :5]).ledoit_wolf()

In [16]:
mu = mean_historical_return(use_df)
S = CovarianceShrinkage(use_df).ledoit_wolf()

In [17]:
mu

3S_Close                  0.451805
AJ네트웍스_Close              0.519111
AK홀딩스_Close               0.000291
APS홀딩스_Close              0.562431
AP시스템_Close               0.589819
                            ...   
히어로즈 TDF2050액티브_Close     0.545218
히어로즈 글로벌리츠이지스액티브_Close    0.541433
히어로즈 단기채권ESG액티브_Close     0.718551
히어로즈 리츠이지스액티브_Close       0.531678
힘스_Close                  0.514963
Length: 3563, dtype: float64

In [19]:
S

,3S_Close,AJ네트웍스_Close,AK홀딩스_Close,APS홀딩스_Close,AP시스템_Close,AP위성_Close,ARIRANG 200_Close,ARIRANG 200동일가중_Close,ARIRANG 200선물레버리지_Close,ARIRANG 200선물인버스2X_Close,...,흥국화재우_Close,흥아해운_Close,희림_Close,히어로즈 TDF2030액티브_Close,히어로즈 TDF2040액티브_Close,히어로즈 TDF2050액티브_Close,히어로즈 글로벌리츠이지스액티브_Close,히어로즈 단기채권ESG액티브_Close,히어로즈 리츠이지스액티브_Close,힘스_Close
3S_Close,1.788180e+11,-7.700235e-01,-1.224986e+00,-1.288780e+00,-3.569218e+00,-0.972346,-1.768668,-0.925628,-1.857730,-1.676128,...,-0.319677,-0.296061,-0.124759,-0.966455,-0.965481,-9.654809e-01,-9.432666e-01,-8.975860e+00,-8.826078e-01,-1.521471e+00
AJ네트웍스_Close,-7.700235e-01,1.788180e+11,-2.599758e+00,-2.773833e+00,-8.202662e+00,-2.042456,-4.021937,-1.900798,-3.974734,-3.760631,...,-0.790813,-0.681085,-0.231256,-1.930282,-1.928336,-1.928337e+00,-1.904104e+00,-1.884108e+01,-1.942671e+00,-3.259891e+00
AK홀딩스_Close,-1.224986e+00,-2.599758e+00,1.788180e+11,1.165143e+04,-1.529266e+01,-3.931771,-7.564580,-3.496278,-7.414197,-7.056197,...,6756.510134,6076.154108,2356.739502,-3.666383,-3.662687,-3.662687e+00,-3.711489e+00,-3.612321e+01,-3.642800e+00,-6.046536e+00
APS홀딩스_Close,-1.288780e+00,-2.773833e+00,1.165143e+04,1.788180e+11,-1.716102e+01,-4.034019,-7.666691,-3.628117,-7.544128,-7.003555,...,3032.143776,2726.802245,1057.582529,-3.807656,-3.803818,-3.803818e+00,-3.800340e+00,-3.804791e+01,-3.807966e+00,-6.308194e+00
AP시스템_Close,-3.569218e+00,-8.202662e+00,-1.529266e+01,-1.716102e+01,1.788183e+11,-12.326492,-23.527351,-10.989201,-23.192899,-22.315068,...,-3.970137,-3.391406,-1.388847,-11.479181,-11.467610,-1.146761e+01,-1.173499e+01,-1.142171e+02,-1.145967e+01,-1.905008e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
히어로즈 TDF2050액티브_Close,-9.654809e-01,-1.928337e+00,-3.662687e+00,-3.803818e+00,-1.146761e+01,-2.987654,-5.617733,-2.659431,-5.558289,-5.254924,...,-0.978476,-0.907364,-0.071195,14552.962667,14538.290862,1.788180e+11,-2.772585e+00,-2.743094e+01,-2.737659e+00,-4.519919e+00
히어로즈 글로벌리츠이지스액티브_Close,-9.432666e-01,-1.904104e+00,-3.711489e+00,-3.800340e+00,-1.173499e+01,-2.994896,-5.743088,-2.693857,-5.659395,-5.480044,...,-1.015589,-0.841272,-0.311882,-2.792335,-2.779253,-2.772585e+00,1.788180e+11,-2.824614e+01,-2.764618e+00,-4.629190e+00
히어로즈 단기채권ESG액티브_Close,-8.975860e+00,-1.884108e+01,-3.612321e+01,-3.804791e+01,-1.142171e+02,-29.703519,-56.264185,-26.458579,-55.119559,-54.082413,...,-9.645495,-8.674189,-2.975625,-27.458611,-27.430934,-2.743094e+01,-2.824614e+01,1.788195e+11,-2.743515e+01,-4.561683e+01
히어로즈 리츠이지스액티브_Close,-8.826078e-01,-1.942671e+00,-3.642800e+00,-3.807966e+00,-1.145967e+01,-2.927988,-5.616666,-2.656561,-5.558199,-5.258184,...,-0.949082,-0.857255,-0.398092,-2.740421,-2.737659,-2.737659e+00,-2.764618e+00,-2.743515e+01,1.788180e+11,-4.559228e+00


In [20]:
from pypfopt.efficient_frontier import EfficientFrontier

ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()

In [21]:
cleaned_weights = ef.clean_weights()
ef.save_weights_to_file("weights.txt")  # saves to file
print(cleaned_weights)

OrderedDict([('3S_Close', 0.0003), ('AJ네트웍스_Close', 0.00034), ('AK홀딩스_Close', 0.0), ('APS홀딩스_Close', 0.00037), ('AP시스템_Close', 0.00039), ('AP위성_Close', 0.00036), ('ARIRANG 200_Close', 0.00042), ('ARIRANG 200동일가중_Close', 0.00036), ('ARIRANG 200선물레버리지_Close', 0.00041), ('ARIRANG 200선물인버스2X_Close', 0.00034), ('ARIRANG ESG가치주액티브_Close', 0.00035), ('ARIRANG ESG성장주액티브_Close', 0.00035), ('ARIRANG ESG우수기업_Close', 0.00035), ('ARIRANG Fn K리츠_Close', 0.00035), ('ARIRANG K-유니콘투자기업액티브_Close', 0.00036), ('ARIRANG KRX300_Close', 0.00038), ('ARIRANG KRX300헬스케어_Close', 0.00035), ('ARIRANG KS로우볼가중TR_Close', 0.00037), ('ARIRANG KS로우사이즈가중TR_Close', 0.00038), ('ARIRANG KS모멘텀가중TR_Close', 0.00037), ('ARIRANG KS밸류가중TR_Close', 0.00038), ('ARIRANG KS퀄리티가중TR_Close', 0.00037), ('ARIRANG S&P글로벌인프라_Close', 0.00038), ('ARIRANG TDF2030액티브_Close', 0.00036), ('ARIRANG TDF2040액티브_Close', 0.00036), ('ARIRANG TDF2050액티브_Close', 0.00036), ('ARIRANG TDF2060액티브_Close', 0.00036), ('ARIRANG iSelect우주항공&UAM_Close', 0.00037), ('

In [22]:
ef.portfolio_performance(verbose=True)

Expected annual return: 53.4%
Annual volatility: 796069.7%
Sharpe Ratio: 0.00


(0.5339785147091365, 7960.696975251923, 6.456451191484665e-05)

In [28]:
stock_df.loc[stock_df.현대비앤지스틸_Close > 1,['Date','현대비앤지스틸_Close']]

,Date,현대비앤지스틸_Close
1265,1998-06-24,4530.0
1266,1998-06-25,4145.0
1267,1998-06-26,4434.0
1268,1998-06-27,4048.0
1269,1998-06-29,4048.0
...,...,...
7261,2022-08-30,14050.0
7262,2022-08-31,14150.0
7263,2022-09-01,13900.0
7264,2022-09-02,13450.0
